# Milestone 3
___

## Introduction
Nama  : Yonathan Anggraiwan

Batch : 028

Program ini dibuat untuk melakukan end-to-end data pipeline dari PostgreSQL ke ElasticSearch. Adapun dataset yang dipakai adalah dataset data karyawan yang didapatkan dari situs [Kaggle](https://www.kaggle.com/datasets/mexwell/employee-performance-and-productivity-data). 

Dalam project kali ini, juga dilakukan validasi data menggunakan teknik Great Expecations, dan visualisasi data yang ditampilkan dalam dashboard interaktif dari Kibana.
___


## Loading Package

In [1]:
import pandas as pd
from great_expectations.data_context import FileDataContext

___
## Loading Clean Data

Dataset yang dipakai adalah dataset data karyawan yang didapatkan dari situs [Kaggle](https://www.kaggle.com/datasets/mexwell/employee-performance-and-productivity-data). Saya akan melakukan eksplorasi lebih lanjut dari dataset ini.

In [5]:
df = pd.read_csv("dags/P2M3_Yonathan_Anggraiwan_data_clean.csv")
df

,employee_id,department,gender,age,job_title,hire_date,years_at_company,education_level,performance_score,monthly_salary,work_hours_per_week,projects_handled,overtime_hours,sick_days,remote_work_frequency,team_size,training_hours,promotions,employee_satisfaction_score,resigned
0,1,IT,Male,55,Specialist,2022-01-19,2,High School,5,6750.0,33,32,22,2,0,14,66,0,2.63,False
1,2,Finance,Male,29,Developer,2024-04-18,0,High School,5,7500.0,34,34,13,14,100,12,61,2,1.72,False
2,3,Finance,Male,55,Specialist,2015-10-26,8,High School,3,5850.0,37,27,6,3,50,10,1,0,3.17,False
3,4,Customer Support,Female,48,Analyst,2016-10-22,7,Bachelor,2,4800.0,52,10,28,12,100,10,0,1,1.86,False
4,5,Engineering,Female,36,Analyst,2021-07-23,3,Bachelor,2,4800.0,38,11,29,13,100,15,9,1,1.25,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,Finance,Male,27,Technician,2022-12-07,1,Bachelor,4,4900.0,55,46,5,3,75,16,48,2,1.28,False
99996,99997,IT,Female,36,Consultant,2018-07-24,6,Master,5,8250.0,39,35,7,0,0,10,77,1,3.48,True
99997,99998,Operations,Male,53,Analyst,2015-11-24,8,High School,2,4800.0,31,13,6,5,0,5,87,1,2.60,False
99998,99999,HR,Female,22,Consultant,2015-08-03,9,High School,5,8250.0,35,43,10,1,75,2,31,1,3.10,False


In [19]:
df.describe()

,employee_id,age,years_at_company,performance_score,monthly_salary,work_hours_per_week,projects_handled,overtime_hours,sick_days,remote_work_frequency,team_size,training_hours,promotions,employee_satisfaction_score
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,50000.500000,41.029410,4.476070,2.995430,6403.211000,44.956950,24.431170,14.514930,7.008550,50.090500,10.013560,49.506060,0.999720,2.999088
std,28867.657797,11.244121,2.869336,1.414726,1372.508717,8.942003,14.469584,8.664026,4.331591,35.351157,5.495405,28.890383,0.815872,1.150719
min,1.000000,22.000000,0.000000,1.000000,3850.000000,30.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,25000.750000,31.000000,2.000000,2.000000,5250.000000,37.000000,12.000000,7.000000,3.000000,25.000000,5.000000,25.000000,0.000000,2.010000
50%,50000.500000,41.000000,4.000000,3.000000,6500.000000,45.000000,24.000000,15.000000,7.000000,50.000000,10.000000,49.000000,1.000000,3.000000
75%,75000.250000,51.000000,7.000000,4.000000,7500.000000,53.000000,37.000000,22.000000,11.000000,75.000000,15.000000,75.000000,2.000000,3.990000
max,100000.000000,60.000000,10.000000,5.000000,9000.000000,60.000000,49.000000,29.000000,14.000000,100.000000,19.000000,99.000000,2.000000,5.000000


___
## Great Expectations Section

Dalam GE (Great Expecations), langkah pertama adalah membuat konteks data yang diolah, lalu menamakan data yang digunakan, serta mengatur lokasi dari data yang diolah, supaya datanya dapat siap untuk divalidasi menggunakan validator Great Expectations.

In [6]:
# Create a data context
context = FileDataContext.create(project_root_dir='./')

In [7]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-data-ms3-1.1'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'ms3'
path_to_data = r'D:\1. Data Science\1. Bootcamp\Fase 2\hck028\p2-ftds028-hck-m3-yonathanggraiwan\dags\P2M3_Yonathan_Anggraiwan_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [8]:
# Creat an expectation suite
expectation_suite_name = 'expectation-data-ms3'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,employee_id,department,gender,age,job_title,hire_date,years_at_company,education_level,performance_score,monthly_salary,work_hours_per_week,projects_handled,overtime_hours,sick_days,remote_work_frequency,team_size,training_hours,promotions,employee_satisfaction_score,resigned
0,1,IT,Male,55,Specialist,2022-01-19,2,High School,5,6750.0,33,32,22,2,0,14,66,0,2.63,False
1,2,Finance,Male,29,Developer,2024-04-18,0,High School,5,7500.0,34,34,13,14,100,12,61,2,1.72,False
2,3,Finance,Male,55,Specialist,2015-10-26,8,High School,3,5850.0,37,27,6,3,50,10,1,0,3.17,False
3,4,Customer Support,Female,48,Analyst,2016-10-22,7,Bachelor,2,4800.0,52,10,28,12,100,10,0,1,1.86,False
4,5,Engineering,Female,36,Analyst,2021-07-23,3,Bachelor,2,4800.0,38,11,29,13,100,15,9,1,1.25,False


Setelah data sudah berhasil disiapkan dan validatornya sudah dihubungkan dengan data, maka kita masuk ke sesi validasinya, dimana saya melakukan 7 ekspektasi dalam data project kali ini.

___
### Expectation 1: To Be Unique

Espektasi 'expect_compound_columns_to_be_unique' digunakan untuk memastikan bahwa gabungan nilai-nilai dari seluruh kolom dalam data tidak ada yang sama persis antar baris. 

Artinya, setiap karyawan memiliki satu kombinasi informasi yang unik seperti "fingerprint" data mereka.

In [10]:
# Expectation 1: 
# Ensure the uniqueness of each employee record by validating the combination of 20 key attributes.

validator.expect_compound_columns_to_be_unique(
    column_list = [
    "employee_id", 
    "department", 
    "gender", 
    "age", 
    "job_title", 
    "hire_date",
    "years_at_company", 
    "education_level", 
    "performance_score", 
    "monthly_salary",
    "work_hours_per_week", 
    "projects_handled", 
    "overtime_hours", 
    "sick_days",
    "remote_work_frequency", 
    "team_size", 
    "training_hours", 
    "promotions",
    "employee_satisfaction_score", 
    "resigned"])

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dari pengecekan, semua gabungan nilai-nilai dari seluruh kolom dalam data tidak ada yang sama persis antar baris. 

- Total pengecekan baris: 10.000 baris
- Nilai duplikat yang tidak terduga: 0
- Nilai hilang: 0
- Persentase tak terduga: 0,0%

___
### Expectation 2: To Be Between

Ekspektasi 'expect_column_values_to_be_between' digunakana untuk memastikan nilai value dalam suatu kolom berada dalam suatu rentan yang ada.

Dalam data ini, saya ingin memvalidasi apakah data gaji bulanan atau 'Monthly_Salary' seluruh karyawan terdapat dalam rentan 3850 hingga 9000.

In [11]:
# Expectation 2:
# Ensure that all values in the 'Monthly_Salary' column fall within a reasonable range,
# between 3,850 and 9,000. This range could represent normalized salary units.

validator.expect_column_values_to_be_between(
    column="monthly_salary",
    min_value=3_850,
    max_value=9000
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dalam pengecekan, dapat disimpulkan kebenaran dari rentan gaji karyawan atau 'Monthly_Salary' berada dalam rentan 3850 hingga 9000.

- Total pengecekan baris: 10.000 baris
- Persentase tak terduga: 0,0%
- Nilai hilang: 0
- Total persentase tak terduga: 0,0%

___
### Expectation 3: Most Common Value To Be In Set

Ekspektasi 'expect_column_most_common_value_to_be_in_set' digunakan untuk memvalidasi nilai atau value data yang paling sering muncul dalam suatu kolom.

Pada ekspektasi ini, saya memvalidasi manakah bagian tim 'IT' atau 'Marketing' yang memiliki paling banyak karyawan dalam perusahaan.

In [12]:
# Expectation 3:
# Confirm that the most frequently occurring value in the 'Department' column is either 'IT' or 'Marketing'. 
# This helps check for dominant categories. If there is a tie between values, it is still considered valid.

validator.expect_column_most_common_value_to_be_in_set(
    column="department",
    value_set=["IT", "Marketing"],
    ties_okay=True  # set ke True kalau kamu mau toleransi nilai dominan lebih dari 1
)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "Marketing"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dalam pengecekan, bagian tim Marketing yang karyawannya terbanyak di perusahaan.

- Nilai obseravasi: Marketing

___
### Expecation 4: Column Values To Be In Set

Ekspektasi 'expect_column_values_to_be_in_set' digunakan untuk melakukan validasi nilai dalam suatu kolom apakah benar berisi nilai-nilai yang divalidasi atau tidak.

Dalam ekspektasi ini, saya memvalidasi apakah hanya terdapat gelar pendidikan 'High School', 'Bachelor', 'Master', dan 'PhD' untuk semua karyawan perusahaan.

In [13]:
# Expectation 4:
# Validate that all values in the 'Education_Level' column match one of the expected options.
# Helps ensure categorical integrity and catch typos or unexpected entries.

validator.expect_column_values_to_be_in_set(
    column="education_level",
    value_set=['High School', 'Bachelor', 'Master', 'PhD']  # Ganti sesuai daftar nilai valid kamu
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dalam pengecekan, betul adanya bahwa hanya terdapat gelar pendidikan 'High School', 'Bachelor', 'Master', dan 'PhD' untuk semua karyawan perusahaan.

- Total pengecekan baris: 10.000 baris
- Persentase tak terduga: 0,0%
- Nilai hilang: 0
- Total persentase tak terduga: 0,0%

___
### Expectation 5: Column Values To Not Be Null

Ekspektasi 'expect_column_values_to_not_be_null' diperlukan untuk mengecek atau memvalidasi apakah terdapat data hilang dalam suatu kolom atau tidak.

Disini saya ingin mengecek apakah semua status 'Resigned' karyawan sudah terisi semua atau belum.

In [14]:
# Expectation 5:
# Check that the 'resigned' column has no null (missing) values.
# Essential when this column is used as a target variable or in reporting.

validator.expect_column_values_to_not_be_null(column="resigned")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dalam pengecekan, betul bahwa semua karyawan sudah memiliki status 'Resigned' masing-masing.

- Total pengecekan baris: 10.000 baris
- Persentase tak terduga: 0,0%

___
### Expectation 6: Table Column Count To Equal

Ekspektasi 'expect_table_column_count_to_equal' digunakan untuk mengecek atau memvalidasi apakah benar jumlah data karyawan yang sedang digunakan, benar berjumlah sekian kolom atau sekian data.

Pada data ini, jumlah data karyawan yang seharusnya dimiliki adalah 20 tipe data atau 20 kolom data.

In [15]:
# Expectation 6:
# Ensure the dataset has exactly 20 columns as expected by the schema.
# This guards against structural changes in the dataset over time.

validator.expect_table_column_count_to_equal(value=20)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 20
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dalam pengecekan, betul bahwa data yang digunakan dalam observasi berjumlah 20 kolom data atau 20 tipe data karyawan.

- Kolom yang diobservasi: 20 kolom

___
### Expectation 7: Column Values To Be Of Type

Ekspektasi 'expect_column_values_to_be_of_type' digunakan disaat ingin memvalidasi tipe value dalam suatu kolom apakah sesuai kriteria atau tidak.

Pada ekspektasi ini, saya ingin memvalidasi apakah data kolom gaji bulanan atau 'Monthly_Salary' berisikan nilai atau value bertipe 'float' atau tidak.

In [16]:
# Expectation 7:
# Validate that values in the 'Monthly_Salary' column are of float data type.
# Important for calculations, aggregations, and preventing type-related bugs.

validator.expect_column_values_to_be_of_type(
    column="monthly_salary",
    type_="float"
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dalam pengecekan, betul adanya bahwa data kolom gaji bulanan atau 'Monthly_Salary' berisikan nilai atau value bertipe 'float'.

- Nilai data yang diobservasi: 'float 64'

Setelah semua validasi data sudah dilakukan, maka selanjutnya adalah melakukan 'checkpoint' atau menyimpan semua ekspektasi yang sudah dilakukan dalam data.

___

## Checkpoint

Checkpoints digunakan untuk:
- Menyisipkan validasi data ke dalam DAG,
- Memastikan kualitas data sebelum diproses lebih lanjut,
- Menyimpan hasil validasi supaya bisa divisualisasikan atau dilihat kembali.

In [ ]:
# Create a checkpoint
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

# Run a checkpoint
checkpoint_result = checkpoint_1.run()

Selanjutnya adalah membuat 'data docs'. Data Docs adalah auto-generated HTML documentation dari hasil validasi yang sudah dijalankan lewat checkpoint atau expectation suite. 

Great Expecation secara otomatis membuat visualisasi hasil validasi data yang lengkap dengan:
- Status expectation (berhasil/gagal)
- Statistik dataset
- Link ke suite dan checkpoint
- Metadata dan batch detail

___

## Data Docs

In [63]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://d:\\1. Data Science\\1. Bootcamp\\Fase 2\\hck028\\p2-ftds028-hck-m3-yonathanggraiwan\\gx\\uncommitted/data_docs/local_site/index.html'}

Sesi great expectation sudah selesai, lanjut ke sesi visualisasi data menggunakan kibana dalam elasticsearch.

___